In [1]:
import pandas as pd
import numpy as np 
import datetime

In [2]:
region = pd.read_csv("RAW/region.csv")
region

,region_id,region_name
0,1,Africa
1,2,America
2,3,Asia
3,4,Europe
4,5,Oceania


In [3]:
customer_nodes = pd.read_csv("RAW/Customer_Nodes.csv")

customer_nodes = customer_nodes.merge(region, 'left', 'region_id')

#removing whitespaces from time_stamps
customer_nodes['end_date'] = customer_nodes['end_date'].str.strip()

customer_nodes


,customer_id,region_id,node_id,start_date,end_date,region_name
0,1,3,4,2020-01-02,2020-01-03,Asia
1,2,3,5,2020-01-03,2020-01-17,Asia
2,3,5,4,2020-01-27,2020-02-18,Oceania
3,4,5,4,2020-01-07,2020-01-19,Oceania
4,5,3,3,2020-01-15,2020-01-23,Asia
...,...,...,...,...,...,...
3495,496,3,4,2020-02-25,9999-12-31,Asia
3496,497,5,4,2020-05-27,9999-12-31,Oceania
3497,498,1,2,2020-04-05,9999-12-31,Africa
3498,499,5,1,2020-02-03,9999-12-31,Oceania


In [4]:
customer_transactions = pd.read_csv("RAW/Customer_Transactions.csv")
customer_transactions

,customer_id,txn_date,txn_type,txn_amount
0,429,2020-01-21,deposit,82
1,155,2020-01-10,deposit,712
2,398,2020-01-01,deposit,196
3,255,2020-01-14,deposit,563
4,185,2020-01-29,deposit,626
...,...,...,...,...
5863,189,2020-02-03,withdrawal,870
5864,189,2020-03-22,purchase,718
5865,189,2020-02-06,purchase,393
5866,189,2020-01-22,deposit,302


A. Customer Nodes Exploration

How many unique nodes are there on the Data Bank system?

What is the number of nodes per region?

How many customers are allocated to each region?

How many days on average are customers reallocated to a different node?

What is the median, 80th and 95th percentile for this same reallocation days metric for each region?

In [5]:
#How many unique nodes are there on the Data Bank system?
print("there are",customer_nodes.node_id.count(),"entries of nodes in the Data bank System")

print("these are all variations of the following ",len(customer_nodes.node_id.unique()),":",customer_nodes.node_id.unique())

print("in the following proportions", customer_nodes.groupby('node_id').size())

there are 3500 entries of nodes in the Data bank System
these are all variations of the following  5 : [4 5 3 1 2]
in the following proportions node_id
1    728
2    662
3    699
4    704
5    707
dtype: int64


In [6]:
#What is the number of nodes per region?
print("The Distribution for nodes per region is ", customer_nodes.groupby('region_name').size())

The Distribution for nodes per region is  region_name
Africa     770
America    735
Asia       714
Europe     665
Oceania    616
dtype: int64


In [7]:
#How many customers are allocated to each region?

print("Customers Allocated Per region are")
customer_nodes.groupby('region_name').agg(customers=('customer_id', 'nunique'))



Customers Allocated Per region are


,customers
region_name,
Africa,110
America,105
Asia,102
Europe,95
Oceania,88


In [8]:
print("The Average Distribution per region is", int(
    customer_nodes.customer_id.nunique()/customer_nodes.region_id.nunique()))


The Average Distribution per region is 100


In [9]:
#How many days on average are customers reallocated to a different node?


#filtering out outlier 9999-12-31
cn_2 = customer_nodes[customer_nodes['end_date'] != '9999-12-31'][['start_date','end_date']].copy()

#parsing dates to datetime from string
cn_2['start_date'] = pd.to_datetime(cn_2['start_date'], format='%Y-%m-%d')
cn_2['end_date'] = pd.to_datetime(cn_2['end_date'], format='%Y-%m-%d')

#calculating time_delta in a list 
time_deltas = cn_2['end_date']-cn_2['start_date']

#calculating average time a customer stays in a node
# giving datetime.timedelta(0) as the start value makes sum work on tds
average_time_delta = sum(time_deltas, datetime.timedelta(0))/len(time_deltas)

print("On average a customer is relocated to a new node after",average_time_delta.days,"days")

#clearing temporary dataframe and lists from memory 
del cn_2
del time_deltas


On average a customer is relocated to a new node after 14 days


In [10]:
#Handling outliers in End_Date

#Parsing Start_date in Datetime format
customer_nodes['start_date'] = pd.to_datetime(customer_nodes['start_date'], format='%Y-%m-%d')

#Replacing Values with date '9999-12-31' with Start + Average Node relocation Time
customer_nodes['end_date'] = np.where((customer_nodes['end_date'] == '9999-12-31'),
                                   (customer_nodes['start_date'] + datetime.timedelta(days=average_time_delta.days)).apply(lambda x: x.strftime("%Y-%m-%d")), customer_nodes['end_date'])

customer_nodes['end_date'] = pd.to_datetime(customer_nodes['end_date'], format='%Y-%m-%d')

#Parsing End_Date in Datetime format
customer_nodes['end_date'] = pd.to_datetime(customer_nodes['end_date'], format='%Y-%m-%d')

customer_nodes


,customer_id,region_id,node_id,start_date,end_date,region_name
0,1,3,4,2020-01-02,2020-01-03,Asia
1,2,3,5,2020-01-03,2020-01-17,Asia
2,3,5,4,2020-01-27,2020-02-18,Oceania
3,4,5,4,2020-01-07,2020-01-19,Oceania
4,5,3,3,2020-01-15,2020-01-23,Asia
...,...,...,...,...,...,...
3495,496,3,4,2020-02-25,2020-03-10,Asia
3496,497,5,4,2020-05-27,2020-06-10,Oceania
3497,498,1,2,2020-04-05,2020-04-19,Africa
3498,499,5,1,2020-02-03,2020-02-17,Oceania


In [11]:
#What is the median, 80th and 95th percentile for this same reallocation days metric for each region?
customer_nodes['time_delta'] = round((customer_nodes['end_date'] - customer_nodes['start_date']).astype('timedelta64[D]'),0)

print("overall median value is",customer_nodes['time_delta'].median())


overall median value is 14.0


In [12]:
def q80(x):
    return x.quantile(0.80)

def q95(x):
    return x.quantile(0.95)

metrics = {'time_delta': ['median', q80, q95]}
customer_nodes.groupby('region_name').agg(metrics)


time_delta            
                median   q80   q95
region_name                       
Africa            14.0  22.0  28.0
America           14.0  22.0  27.0
Asia              14.0  23.0  28.0
Europe            14.0  22.0  28.0
Oceania           14.0  23.0  28.0

--Same Solution in PostgresSQL

How many unique nodes are there on the Data Bank system?
What is the number of nodes per region?
How many customers are allocated to each region?
How many days on average are customers reallocated to a different node?
What is the median, 80th and 95th percentile for this same reallocation days metric for each region?


--How many unique nodes are there on the Data Bank system?
SELECT COUNT(DISTINCT node_id) AS UNIQUE_NODES
FROM data_bank.customer_nodes


--What is the number of nodes per region?
SELECT region_name,COUNT(node_id) As node_frequecy 
FROM data_bank.customer_nodes
LEFT JOIN data_bank.regions 
ON data_bank.customer_nodes.region_id = data_bank.regions.region_id
GROUP BY region_name
ORDER BY 2 DESC


--How many customers are allocated to each region?
SELECT region_name,COUNT(DISTINCT customer_id) As customer_frequecy 
FROM data_bank.customer_nodes
LEFT JOIN data_bank.regions 
ON data_bank.customer_nodes.region_id = data_bank.regions.region_id
GROUP BY region_name
ORDER BY 2 DESC


--How many days on average are customers reallocated to a different node?

SELECT SUM(end_date-start_date)/COUNT(*) as delta
FROM data_bank.customer_nodes
WHERE end_date <> '9999-12-31'

--What is the median, 80th and 95th percentile for this same reallocation days metric for each region?

WITH Average_Date AS (
  SELECT SUM(end_date-start_date)/COUNT(*) as Average_Date
  FROM data_bank.customer_nodes
  WHERE end_date <> '9999-12-31'
)

,customer_nodes_update AS (
  SELECT customer_id,region_id,node_id,start_date
  ,CASE WHEN end_date = '9999-12-31' THEN start_date + (SELECT Average_Date From Average_Date) * INTERVAL '1 day' ELSE end_date END AS end_date
  FROM data_bank.customer_nodes
)

SELECT 
 EXTRACT(DAY FROM PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY end_date-start_date)) AS median
 ,EXTRACT(DAY FROM PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY end_date-start_date)) AS p_80
 ,EXTRACT(DAY FROM PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY end_date-start_date)) AS P_95
  FROM customer_nodes_update




B. Customer Transactions

What is the unique count and total amount for each transaction type?

What is the average total historical deposit counts and amounts for all customers?

For each month - how many Data Bank customers make more than 1 deposit and either 1 purchase or 1 withdrawal in a single month?

What is the closing balance for each customer at the end of the month?

What is the percentage of customers who increase their closing balance by more than 5%?


In [77]:
customer_transactions = pd.read_csv("RAW/Customer_Transactions.csv")
#removing whitespaces from txn_type
customer_transactions['txn_type'] = customer_transactions['txn_type'].str.strip()
customer_transactions


,customer_id,txn_date,txn_type,txn_amount
0,429,2020-01-21,deposit,82
1,155,2020-01-10,deposit,712
2,398,2020-01-01,deposit,196
3,255,2020-01-14,deposit,563
4,185,2020-01-29,deposit,626
...,...,...,...,...
5863,189,2020-02-03,withdrawal,870
5864,189,2020-03-22,purchase,718
5865,189,2020-02-06,purchase,393
5866,189,2020-01-22,deposit,302


In [32]:
#What is the unique count and total amount for each transaction type?
customer_transactions.groupby('txn_type').agg(unique_count = pd.NamedAgg(column='txn_type',aggfunc='count'),total_sum = pd.NamedAgg(column='txn_amount',aggfunc='sum'))

,unique_count,total_sum
txn_type,,
deposit,2671,1359168
purchase,1617,806537
withdrawal,1580,793003


In [41]:
#What is the average total historical deposit counts and amounts for all customers?
customer_transactions[customer_transactions.txn_type == "deposit"].txn_date.min(),customer_transactions[customer_transactions.txn_type == "deposit"].txn_date.max()


(' 2020-01-01', ' 2020-04-26')

In [70]:
#What is the average total historical deposit counts and amounts for all customers?
print("average total deposit for each customer is",round(customer_transactions[customer_transactions.txn_type == "deposit"].customer_id.sum(
)/customer_transactions[customer_transactions.txn_type == "deposit"].customer_id.count()))

print(round(customer_transactions[customer_transactions.txn_type == "deposit"].txn_amount.count(
) / customer_transactions.customer_id.nunique()), "deposits are made on average by each customer")


average total deposit for each customer is 252
5 deposits are made on average by each customer


In [94]:
#For each month - how many Data Bank customers make more than 1 deposit and either 1 purchase or 1 withdrawal in a single month?


customer_transactions['month'] = pd.DatetimeIndex(pd.to_datetime(customer_transactions['txn_date'], format='%Y-%m-%d')).month
customer_transactions['year'] = pd.DatetimeIndex(pd.to_datetime(customer_transactions['txn_date'], format='%Y-%m-%d')).year


In [169]:
#customer_transactions
customer_transactions['tosum'] =1
customer_transactions

,customer_id,txn_date,txn_type,txn_amount,month,year,tosum
0,429,2020-01-21,deposit,82,1,2020,1
1,155,2020-01-10,deposit,712,1,2020,1
2,398,2020-01-01,deposit,196,1,2020,1
3,255,2020-01-14,deposit,563,1,2020,1
4,185,2020-01-29,deposit,626,1,2020,1
...,...,...,...,...,...,...,...
5863,189,2020-02-03,withdrawal,870,2,2020,1
5864,189,2020-03-22,purchase,718,3,2020,1
5865,189,2020-02-06,purchase,393,2,2020,1
5866,189,2020-01-22,deposit,302,1,2020,1


In [168]:
pd.crosstab(customer_transactions["customer_id"], customer_transactions.txn_type)


txn_type,deposit,purchase,withdrawal
customer_id,,,
1,2,2,0
2,2,0,0
3,2,1,2
4,2,1,0
5,4,3,4
...,...,...,...
496,2,1,3
497,4,2,0
498,8,1,0


In [175]:
customer_transactions_monthly_count_df= customer_transactions.pivot_table(
    index=["year", "month", "customer_id"] , columns='txn_type' ,values='tosum', aggfunc='sum', fill_value=0)
    
#customer_transactions_monthly_count_df.columns = customer_transactions_monthly_count_df.columns.droplevel(0)  # remove amount
customer_transactions_monthly_count_df.columns.name = None  # remove categories
customer_transactions_monthly_count_df = customer_transactions_monthly_count_df.reset_index()  # index to columns

#customer_transactions_monthly_count_df
customer_transactions_monthly_count_df


,year,month,customer_id,deposit,purchase,withdrawal
0,2020,1,1,1,0,0
1,2020,1,2,1,0,0
2,2020,1,3,1,0,0
3,2020,1,4,2,0,0
4,2020,1,5,2,0,1
...,...,...,...,...,...,...
1715,2020,4,489,4,1,2
1716,2020,4,490,0,1,0
1717,2020,4,493,1,0,1
1718,2020,4,497,1,1,0


In [194]:
customer_transactions_monthly_count_filtered_df = customer_transactions_monthly_count_df[(customer_transactions_monthly_count_df.deposit >= 1) & (
    (customer_transactions_monthly_count_df.purchase == 1) | (customer_transactions_monthly_count_df.withdrawal == 1))]
customer_transactions_monthly_count_filtered_df


,year,month,customer_id,deposit,purchase,withdrawal
4,2020,1,5,2,0,1
7,2020,1,8,2,1,0
11,2020,1,12,1,0,1
12,2020,1,13,3,1,1
15,2020,1,16,3,4,1
...,...,...,...,...,...,...
1711,2020,4,480,1,1,1
1713,2020,4,487,1,1,0
1715,2020,4,489,4,1,2
1717,2020,4,493,1,0,1


In [201]:
customer_transactions_monthly_count_filtered_df.groupby(['year','month']).agg(unique_customer_count=('customer_id', 'nunique')).reset_index()


,year,month,unique_customer_count
0,2020,1,212
1,2020,2,199
2,2020,3,203
3,2020,4,106


In [ ]:
#What is the percentage of customers who increase their closing balance by more than 5%?
